In [1]:
from my_import import *
df_train = pd.read_csv('df_train.csv')
df_val = pd.read_csv('df_val.csv')
df_test = pd.read_csv('df_test.csv')

#Make sure the genre collumns is in lists not strings
#NEED TO DO THIS EVERYTIME EXPORT DATASET
df_train['genres'] = df_train['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_val['genres'] = df_val['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_test['genres'] = df_test['genres'].apply(lambda x: list(ast.literal_eval(x)))

display(df_train)
display(df_val)
display(df_test)

c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,title,synopsis,genres,index
0,Sabaku no Kaizoku! Captain Kuppa,"Sometime in the future, the world was complete...","[Action, Adventure]",1
1,On Air Dekinai!,"Set in 2014, the anime follows the adventures ...",[Comedy],2
2,Panpaka Pants O-New!,Follows a pig whose family's mission is to col...,"[Comedy, Kids]",8
3,Zhu Zhu Xia: Jing Qiu Xiao Yingxiong,"In honor of the 2018 World Cup, this season of...","[Kids, Sci-Fi, Sports, Super Power]",9
4,Fuwafuwa Hour: Pui Pui & Muu Muu,"Fairies living in a fluffy forest, where both ...","[Fantasy, Kids]",10
...,...,...,...,...
9189,Naruto: Dai Katsugeki!! Yuki Hime Shinobu Houj...,The Konohagakure Grand Sports Festival has beg...,"[Action, Comedy, Fantasy, Other, Shounen, Sports]",13373
9190,Grisaia no Meikyuu: Caprice no Mayu 0 - Takizo...,Special bundled with the Blu-ray/DVD volume of .,[Ecchi],13375
9191,Bikini Warriors Special,"According to the official Hobby Japan website,...","[Comedy, Ecchi, Fantasy, Parody]",13376
9192,Re:Zero kara Hajimeru Break Time,A series of comedic shorts featuring chibi ver...,"[Adventure, Comedy, Fantasy, Parody]",13377


,title,synopsis,genres,index
0,Touma Kishinden Oni,Shuramaru is hated and feared by the villagers...,[Supernatural],0
1,Uchi no 3 Shimai,The daily life of the Motsumoto family. The th...,"[Comedy, Slice of Life]",3
2,Running Man,"The Soul Tree, the great source of our race. ...","[Action, Adventure, Kids]",4
3,Miracle! Mimika,Himeno Mimika is a daughter of historical cook...,"[Kids, Other, Slice of Life]",14
4,Kick & Slide,The third generation of J SOUL BROTHERS from E...,"[Kids, Music]",22
...,...,...,...,...
1134,Hyakujitsu no Bara: Jinginaki Nikukyuu-hen,Specials included in the OVA of Hyakujitsu no ...,"[Other, Parody]",13166
1135,Ladies versus Butlers! Specials,DVD/BD specials featuring two female cast memb...,"[Ecchi, Seinen]",13214
1136,Cross Road,"As college entrance examinations draw near, tw...","[School, Slice of Life]",13276
1137,Getsuyoubi no Tawawa Specials,Two special episodes of included on Blu-ray. 1...,"[Ecchi, Slice of Life]",13313


,title,synopsis,genres,index
0,RPG Fudousan,The story begins 15 years after the demon king...,"[Comedy, Fantasy, Slice of Life]",20
1,Hyouga Senshi Gaislugger,A cyborg warrior from an ancient Antarctic kin...,"[Action, Mecha, Sci-Fi]",23
2,Hyper Positive Yogoremon,"The story centers on Otarou, a creature themed...",[Comedy],24
3,Totsugeki! Pappara-tai,"An elite soldier, whose nickname is God of Dea...","[Action, Adventure, Comedy, Romance, Sci-Fi]",31
4,Pink Lady Monogatari: Eikou no Tenshi-tachi,"Based on the real life-story of Pink Lady, a f...","[Drama, Music, Slice of Life]",36
...,...,...,...,...
1122,Dimension W: Short Track/Robot wa Sentou no Yu...,An unaired episode released with the sixth BD ...,"[Sci-Fi, Seinen]",12769
1123,Amagami SS+ Plus: Extra Episode+ Plus,Short extras of the anime included in each of ...,[Romance],12909
1124,Dimension W: W no Tobira Online - Rose no Onay...,Short specials included in the Blu-ray release...,"[Sci-Fi, Seinen]",12959
1125,Sugar*Soldier,Three episodes aired on TV as part of the Ribo...,[Shoujo],13264


In [2]:
import gensim.downloader as api
word2vec = api.load("word2vec-google-news-300")

In [3]:
def text_to_vector(text, model, dim=300):
    words = text.split()
    vectors = [model[word] for word in words if word in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(dim)  # fallback for empty or unknown

X_train_vecs = np.vstack([text_to_vector(s, word2vec) for s in df_train['synopsis']])
X_val_vecs = np.vstack([text_to_vector(s, word2vec) for s in df_val['synopsis']])
X_test_vecs = np.vstack([text_to_vector(s, word2vec) for s in df_test['synopsis']])


mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(df_train['genres'])
y_val = mlb.transform(df_val['genres'])
y_test = mlb.transform(df_test['genres'])

from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier

clf = OneVsRestClassifier(XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
clf.fit(X_train_vecs, y_train)

# Predict probabilities
y_probs_val = clf.predict_proba(X_val_vecs)

c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\xgboost\training.py:183: UserWarning: [01:17:08] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\xgboost\training.py:183: UserWarning: [01:17:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\xgboost\training.py:183: UserWarning: [01:17:15] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\xgboost\training.py:183: UserWarning: [01:17:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_

In [4]:
def optimal_threshold(y_true, y_prob):
    thresholds = np.linspace(0.01, 0.99, 99)
    best_thresh = 0.5
    best_loss = float('inf')
    
    for t in thresholds:
        false_neg = np.sum((y_true == 1) & (y_prob < t))
        false_pos = np.sum((y_true == 0) & (y_prob >= t))
        loss = false_neg + false_pos
        if loss < best_loss:
            best_loss = loss
            best_thresh = t
    return best_thresh
sample_thresholds = np.array([optimal_threshold(y, p) for y, p in zip(y_train, clf.predict_proba(X_train_vecs))])


In [5]:
from xgboost import XGBRegressor

# Use predicted probs as features (i.e., 20-d vector per movie)
X_thresh_train = clf.predict_proba(X_train_vecs)
regressor = XGBRegressor()
regressor.fit(X_thresh_train, sample_thresholds)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [6]:
# Get predicted probabilities for test
y_probs_test = clf.predict_proba(X_test_vecs)

# Predict threshold for each test sample
test_thresholds = regressor.predict(y_probs_test)

# Apply threshold to get final binary predictions
y_pred_test = np.array([
    (sample >= t).astype(int)
    for sample, t in zip(y_probs_test, test_thresholds)
])


In [7]:
from sklearn.metrics import classification_report, jaccard_score, f1_score, roc_auc_score

print(classification_report(y_test, y_pred_test, target_names=mlb.classes_, zero_division=0))
print("Jaccard (samples):", jaccard_score(y_test, y_pred_test, average='samples'))
print("F1-score (samples):", f1_score(y_test, y_pred_test, average='samples'))
hit_rate = (np.logical_and(y_test, y_pred_test).sum(axis=1) > 0).mean()
print(hit_rate)
print("ROC_AUC score:",roc_auc_score(y_true=y_test,y_score=y_pred_test,average="samples"))


               precision    recall  f1-score   support

       Action       0.58      0.72      0.64       316
    Adventure       0.45      0.49      0.47       210
       Comedy       0.53      0.73      0.62       443
       Demons       0.45      0.12      0.19        41
        Drama       0.33      0.34      0.34       198
        Ecchi       0.30      0.05      0.08        63
      Fantasy       0.48      0.60      0.53       235
        Harem       0.50      0.06      0.10        36
       Hentai       0.64      0.46      0.53       100
   Historical       0.59      0.24      0.35        78
       Horror       0.29      0.06      0.10        35
         Kids       0.56      0.26      0.35        89
        Mecha       0.56      0.38      0.45        78
     Military       0.62      0.31      0.42        48
        Music       0.64      0.24      0.35        59
      Mystery       0.50      0.22      0.31        76
        Other       0.33      0.39      0.36       241
       Pa